In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

## IO markup

### Train

In [2]:
from modules.data import bert_data

In [3]:
train_df_path = "/home/eartemov/ae/work/factRuEval-2016/dev.csv"
valid_df_path = "/home/eartemov/ae/work/factRuEval-2016/test.csv"

In [4]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels4.txt",
    clear_cache=True
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
from modules.models.bert_models import BERTNCRF

In [6]:
model = BERTNCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3, nbest=len(data.train_ds.idx2label)-1)

build CRF...


In [7]:
from modules.train.train import NerLearner

In [8]:
num_epochs = 100

In [9]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTNCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [10]:
model.get_n_trainable_params()

299259

In [ ]:
learner.fit(epochs=num_epochs)

### Eval

#### Load model

In [12]:
from modules.data import bert_data

In [13]:
data = bert_data.LearnData.create(
    train_df_path=train_df_path,
    valid_df_path=valid_df_path,
    idx2labels_path="/home/eartemov/ae/work/factRuEval-2016/idx2labels3.txt",
    clear_cache=False
)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /home/eartemov/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729


In [15]:
from modules.models.bert_models import BERTNCRF

In [16]:
model = BERTNCRF.create(len(data.train_ds.idx2label), crf_dropout=0.3, nbest=len(data.train_ds.idx2label)-1)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmpclku_2kv
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "

build CRF...


In [17]:
from modules.train.train import NerLearner

In [18]:
num_epochs = 100

In [19]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/fre-BERTNCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [20]:
model.get_n_trainable_params()

299259

In [21]:
learner.load_model()

### Predict

In [22]:
from modules.data.bert_data import get_data_loader_for_predict

In [23]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [24]:
preds = learner.predict(dl)

In [25]:
from sklearn_crfsuite.metrics import flat_classification_report

In [26]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [27]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [28]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=data.train_ds.idx2label[5:], digits=4)

In [29]:
print(tokens_report)

              precision    recall  f1-score   support

       I_LOC     0.8939    0.7848    0.8358      1557
       I_PER     0.9545    0.9645    0.9595      2112
       I_ORG     0.7930    0.7780    0.7854      3865

   micro avg     0.8592    0.8317    0.8452      7534
   macro avg     0.8805    0.8424    0.8603      7534
weighted avg     0.8591    0.8317    0.8446      7534



In [30]:
from modules.analyze_utils.plot_metrics import get_bert_span_report
from modules.analyze_utils.utils import voting_choicer, tokens2spans

In [31]:
print(get_bert_span_report(dl, preds, fn=voting_choicer))

              precision    recall  f1-score   support

         LOC     0.7760    0.6938    0.7326      1303
           O     0.9726    0.9814    0.9770     48636
         ORG     0.6089    0.5300    0.5667      1868
         PER     0.8567    0.8185    0.8371      1300

    accuracy                         0.9545     53107
   macro avg     0.8035    0.7559    0.7783     53107
weighted avg     0.9522    0.9545    0.9531     53107

